In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Add custom layers for our task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1)(x)

# Create the combined model
model_classification = Model(inputs=base_model.input, outputs=predictions)

# Compile the model (for classification task)
model_classification.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Example dataset (replace this with your labeled dataset)
X_images = [np.random.rand(64, 64, 3) for _ in range(100)]  # Dummy images
Y_calories = np.random.randint(100, 500, size=100)  # Dummy calorie values

# Split the dataset into training and testing sets
X_train, X_test, Y_calories_train, Y_calories_test = train_test_split(X_images, Y_calories, test_size=0.2, random_state=42)

# Train the classification model
model_classification.fit(np.array(X_train), np.array(Y_calories_train), epochs=5, batch_size=32, validation_split=0.2)

# Extract features from the pre-trained MobileNetV2 model
features_train = base_model.predict(np.array(X_train))
features_test = base_model.predict(np.array(X_test))

# Flatten the features
features_train_flat = features_train.reshape(features_train.shape[0], -1)
features_test_flat = features_test.reshape(features_test.shape[0], -1)

# Train a linear regression model for calorie estimation
model_regression = LinearRegression()
model_regression.fit(features_train_flat, Y_calories_train)

# Predict calories using the classification model and regression model
predictions_classification = model_classification.predict(np.array(X_test))
predictions_regression = model_regression.predict(features_test_flat)

# Combine predictions (you might want to fine-tune this based on your needs)
final_predictions = predictions_classification.flatten() * predictions_regression.flatten()

# Ensure both arrays have the same number of samples
min_samples = min(Y_calories_test.shape[0], final_predictions.shape[0])
Y_calories_test_flat = Y_calories_test[:min_samples].flatten()
final_predictions_flat = final_predictions[:min_samples]

# Calculate mean squared error
mse = mean_squared_error(Y_calories_test_flat, final_predictions_flat)
print(f'Mean Squared Error: {mse}')






Epoch 1/5


2/2 [==============================] - 12s 2s/step - loss: 101506.5781 - mae: 295.1856 - val_loss: 85706.6953 - val_mae: 275.4497
Epoch 2/5
2/2 [==============================] - 1s 332ms/step - loss: 88400.8906 - mae: 275.3212 - val_loss: 28575.4609 - val_mae: 148.4519
Epoch 3/5
2/2 [==============================] - 1s 411ms/step - loss: 70609.5703 - mae: 248.8931 - val_loss: 38071.9102 - val_mae: 161.0542
Epoch 4/5
2/2 [==============================] - 1s 335ms/step - loss: 53828.6602 - mae: 214.4511 - val_loss: 39554.8281 - val_mae: 157.0818
Epoch 5/5
1/1 [==============================] - 1s 746ms/step
Mean Squared Error: 3792250129.3615594
